In [6]:
# first import the Model class from docplex.mp
from docplex.mp.model import Model
import numpy as np
import time
import sys

In [4]:
# "jobs" is a matrix where each value is the working time needed for each job on each machine
# jobs[i][j] is the time needed for job j on machine i

def create_job_scheduling(jobs):
    nb_jobs = len(jobs[0])
    M = len(jobs)
    
    # create one model instance, with a name
    mdl = Model(name='job_scheduling_v1')
    
    # Creation of the variables
    # Variable name format: x_<numMachine>_<numJob>_<jobPosition>
    # "x_3_2_4 = 1" ===> "the job #2 will be the 4th job to run on the machine #3"
    # (indexes start at 1)
    x = []
    for m in range(M):
        x.append([])
        for j in range(nb_jobs):
            x[m].append([])
            for k in range(nb_jobs):
                x[m][j].append(mdl.binary_var(name="x_%d_%d_%d" % (m+1, j+1, k+1)))
       
    x = np.asarray(x)
    
    # Each job must be processed one time
    for i in range(nb_jobs):
        mdl.add_constraint(np.sum(np.sum(x[:,i,:])) == 1)
        
    # Each machine can only run 1 job at a time
    for m in range(M):
        for i in range(nb_jobs):
            mdl.add_constraint(np.sum(np.sum(x[m,:,i])) <= 1)
    
    # The machines are running in parallel, so the objective is
    # to minimize the working time of the machine that will work longer than the others
    z = mdl.continuous_var(name="objective")
    for m in range(M):
        mdl.add_constraint(z >= np.sum(np.dot(np.asarray(jobs[m]), x[m,:,:])))
    
    mdl.minimize(z)
    return mdl, x

# remplacer tous les print par des écritures dans un fichier
def format_solution(num, jobs, var, sol, tps_exec): 
    M = len(jobs)
    J = len(jobs[0])
    
    file = open("outputTestFile.txt", "a")
    print(num, file=file)
    print("---", file=file)
    
    for m in range(M):
        start_time = 0
        for p in range(J):
            for j in range(J):
                if sol.get_value(var[m,j,p]) == 1:
                    print("%d,%d,%d,%d" % (m, j, start_time, start_time + jobs[m][j]), file=file)
                    start_time += jobs[m][j]
                    break
    print("---", file=file)
    print("%lf,%d\n" % (tps_exec, sol.get_objective_value()), file=file)

    
def solve_job_scheduling(num, jobs):
    mdl, x = create_job_scheduling(jobs)
    
    t = time.time()
    sol = mdl.solve()
    tps_exec = time.time() - t
    
    if sol:
        #print(sol)
        format_solution(num, jobs, x, sol, tps_exec)    

In [5]:
jobs = [[1, 2, 3, 4, 5, 6],
        [2, 1, 3, 4, 5, 6],
        [2, 3, 1, 4, 5, 6]]
solve_job_scheduling(0, jobs)

In [6]:
jobs = [[4, 5, 6, 7, 4, 3],
        [3, 8, 4, 13, 7, 1],
        [10, 5, 7, 8, 14, 6]]
solve_job_scheduling(1, jobs)

In [23]:
jobs = [[9,8,0,3,4,10,10,10,3,2],
        [5,4,7,0,8,2,0,10,3,9],
        [7,0,2,9,5,1,4,5,2,8],
        [10,8,1,6,0,5,5,3,2,4],
        [2,8,4,4,7,2,9,3,7,6],
        [8,1,2,9,3,9,9,0,1,7],
        [9,4,1,3,3,7,7,1,7,10],
        [5,9,7,4,8,9,3,1,0,2],
        [9,6,2,7,1,6,3,9,0,6],
        [7,8,3,3,7,3,2,2,4,4]]
print(jobs)
solve_job_scheduling(0, jobs)

[[9, 8, 0, 3, 4, 10, 10, 10, 3, 2], [5, 4, 7, 0, 8, 2, 0, 10, 3, 9], [7, 0, 2, 9, 5, 1, 4, 5, 2, 8], [10, 8, 1, 6, 0, 5, 5, 3, 2, 4], [2, 8, 4, 4, 7, 2, 9, 3, 7, 6], [8, 1, 2, 9, 3, 9, 9, 0, 1, 7], [9, 4, 1, 3, 3, 7, 7, 1, 7, 10], [5, 9, 7, 4, 8, 9, 3, 1, 0, 2], [9, 6, 2, 7, 1, 6, 3, 9, 0, 6], [7, 8, 3, 3, 7, 3, 2, 2, 4, 4]]


In [31]:
import csv
def test_job_scheduling(testfile):
    with open(testfile, newline='') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        jobs = []
        num = 0
        
        for row in csvreader:
            if row:
                values = []
                for value in row:
                    values.append(int(value))
                jobs.append(values)
            else:
                #print(jobs)
                #print()
                solve_job_scheduling(num, jobs)
                num += 1
                jobs = []
                

In [33]:
test_job_scheduling("tests.csv")

In [24]:
# 
# PAS IMPORTANT
#

# "jobs" is a matrix where each value is the working time needed for each job on each machine
# jobs[i][j] is the time needed for job j on machine i

def create_job_scheduling_v2(jobs):
    J = len(jobs[0])
    M = len(jobs)
    K = 100000000 # a big number
    
    # create one model instance, with a name
    mdl = Model(name='job_scheduling_v1')
    
    # Creation of the variables
    
        
    # Variable name format: Y_<numMachine>_<numJob>_<jobPosition>
    # "Y_3_2_4 = 1" ===> "job #2 runs before job #4 on machine #3"
    # (indexes start at 1)
    Y = []
    for m in range(M):
        Y.append([])
        for j in range(J):
            Y[m].append([])
            for k in range(J):
                Y[m][j].append(mdl.binary_var(name="Y_%d_%d_%d" % (m+1, j+1, k+1)))
                
    # X[j][m] = starting time of job j on machine m 
    X = []
    for j in range(J):
        X.append([])
        for m in range(M):
            X[j].append(mdl.integer_var(0, K, name="X_%d_%d" % (j+1, m+1)))
    
    Y = np.asarray(Y)
    X = np.asarray(X)
    
        
    # Each machine can only run 1 job at a time
    for m in range(M):
        for i in range(J):
            for j in range(J):
                if i != j:
                    mdl.add_constraint(X[i,m] >= X[j,m] + jobs[m][j] - K * Y[m,i,j])
                mdl.add_constraint(X[j,m] >= X[i,m] + jobs[m][i] - K * (1 - Y[m,i,j]))  
    
    # The machines are running in parallel, so the objective is
    # to minimize the working time of the machine that will work longer than the others
    z = mdl.continuous_var(name="objective")
    for m in range(M):
        for j in range(J):
            mdl.add_constraint(z >= X[j,m] + jobs[m][j])
    
    mdl.minimize(z)
    return mdl, X

def solve_job_scheduling_v2(jobs):
    mdl, x = create_job_scheduling_v2(jobs)
    sol = mdl.solve()
    if sol:
        print(sol)
        for i in range(len(jobs[0])):
            print(sol.get_values(x[i]))

In [25]:
jobs = [[1, 2, 3, 4, 5, 6],
        [2, 1, 3, 4, 5, 6],
        [2, 3, 1, 4, 5, 6]]
solve_job_scheduling_v2(jobs)

solution for: job_scheduling_v1
objective: 6
Y_1_1_2=0
Y_1_1_3=0
Y_1_1_4=0
Y_1_1_5=0
Y_1_1_6=0
Y_1_2_1=0
Y_1_2_3=0
Y_1_2_4=0
Y_1_2_5=0
Y_1_2_6=0
Y_1_3_1=0
Y_1_3_2=0
Y_1_3_4=0
Y_1_3_5=0
Y_1_3_6=0
Y_1_4_1=0
Y_1_4_2=0
Y_1_4_3=0
Y_1_4_5=0
Y_1_4_6=0
Y_1_5_1=0
Y_1_5_2=0
Y_1_5_3=0
Y_1_5_4=0
Y_1_5_6=0
Y_1_6_1=0
Y_1_6_2=0
Y_1_6_3=0
Y_1_6_4=0
Y_1_6_5=0
Y_2_1_2=0
Y_2_1_3=0
Y_2_1_4=0
Y_2_1_5=0
Y_2_1_6=0
Y_2_2_1=0
Y_2_2_3=0
Y_2_2_4=0
Y_2_2_5=0
Y_2_2_6=0
Y_2_3_1=0
Y_2_3_2=0
Y_2_3_4=0
Y_2_3_5=0
Y_2_3_6=0
Y_2_4_1=0
Y_2_4_2=0
Y_2_4_3=0
Y_2_4_5=0
Y_2_4_6=0
Y_2_5_1=0
Y_2_5_2=0
Y_2_5_3=0
Y_2_5_4=0
Y_2_5_6=0
Y_2_6_1=0
Y_2_6_2=0
Y_2_6_3=0
Y_2_6_4=0
Y_2_6_5=0
Y_3_1_2=0
Y_3_1_3=0
Y_3_1_4=0
Y_3_1_5=0
Y_3_1_6=0
Y_3_2_1=0
Y_3_2_3=0
Y_3_2_4=0
Y_3_2_5=0
Y_3_2_6=0
Y_3_3_1=0
Y_3_3_2=0
Y_3_3_4=0
Y_3_3_5=0
Y_3_3_6=0
Y_3_4_1=0
Y_3_4_2=0
Y_3_4_3=0
Y_3_4_5=0
Y_3_4_6=0
Y_3_5_1=0
Y_3_5_2=0
Y_3_5_3=0
Y_3_5_4=0
Y_3_5_6=0
Y_3_6_1=0
Y_3_6_2=0
Y_3_6_3=0
Y_3_6_4=0
Y_3_6_5=0
objective=6.000

[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0